In [1]:
import src.setcwd

src.setcwd.main()

Working directory:  /home/adele/code/my28brains/src
Directory added to path:  /home/adele/code/my28brains
Directory added to path:  /home/adele/code/my28brains/src
Directory added to path:  /home/adele/code/my28brains/H2_SurfaceMatch
Directory added to path:  /home/adele/code/my28brains/src/notebooks
Directory added to path:  /home/adele/code/my28brains/src/notebooks/csv


In [2]:
import os
os.environ["GEOMSTATS_BACKEND"] = "pytorch"  # noqa: E402
import geomstats.backend as gs

import datasets.utils as data_utils

from src.regression import training

INFO: Using pytorch backend


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
null = 0
false = False
true = True

raw_config = {
  "n_X": {
    "desc": null,
    "value": 20
  },
  "_wandb": {
    "desc": null,
    "value": {
      "t": {
        "1": [
          1,
          5,
          53,
          55
        ],
        "2": [
          1,
          5,
          53,
          55
        ],
        "3": [
          2,
          15,
          17,
          23,
          37
        ],
        "4": "3.10.12",
        "5": "0.15.8",
        "8": [
          5
        ]
      },
      "framework": "torch",
      "start_time": 1699033550.387635,
      "cli_version": "0.15.8",
      "is_jupyter_run": false,
      "python_version": "3.10.12",
      "is_kaggle_kernel": false
    }
  },
  "full_run": {
    "desc": null,
    "value": false
  },
  "tol_factor": {
    "desc": null,
    "value": 0.1
  },
  "dataset_name": {
    "desc": null,
    "value": "hyperboloid"
  },
  "linear_noise": {
    "desc": null,
    "value": false
  },
  "noise_factor": {
    "desc": null,
    "value": 0
  },
  "space_dimension": {
    "desc": null,
    "value": 5
  },
  "linear_residuals": {
    "desc": null,
    "value": false
  },
  "project_linear_noise": {
    "desc": null,
    "value": false
  },
  "geodesic_initialization": {
    "desc": null,
    "value": "warm_start"
  }
}

In [4]:
class AttrDict(dict):
    """Convert a dict into an object where attributes are accessed with "."

    This is needed for the utils.load() function.
    """

    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self


In [5]:
config = {}
for key, value in raw_config.items():
    config[key] = value["value"]
config["dataset_name"]

config = AttrDict(config)

In [6]:
(
    space,
    y,
    y_noiseless,
    X,
    true_intercept,
    true_coef,
) = data_utils.load(config)

Creating synthetic dataset on hyperboloid
intercept shape: torch.Size([6])
coef shape: torch.Size([6])
--> In hyperboloid.py, exp
tangent_vec tensor([[-1.0718, -0.5606, -0.5599, -0.6357, -0.5846, -0.5069],
        [-0.9590, -0.5016, -0.5009, -0.5688, -0.5230, -0.4536],
        [-0.8462, -0.4426, -0.4420, -0.5019, -0.4615, -0.4002],
        [-0.7334, -0.3835, -0.3831, -0.4350, -0.4000, -0.3469],
        [-0.6205, -0.3245, -0.3241, -0.3681, -0.3384, -0.2935],
        [-0.5077, -0.2655, -0.2652, -0.3011, -0.2769, -0.2401],
        [-0.3949, -0.2065, -0.2063, -0.2342, -0.2154, -0.1868],
        [-0.2821, -0.1475, -0.1473, -0.1673, -0.1538, -0.1334],
        [-0.1692, -0.0885, -0.0884, -0.1004, -0.0923, -0.0800],
        [-0.0564, -0.0295, -0.0295, -0.0335, -0.0308, -0.0267],
        [ 0.0564,  0.0295,  0.0295,  0.0335,  0.0308,  0.0267],
        [ 0.1692,  0.0885,  0.0884,  0.1004,  0.0923,  0.0800],
        [ 0.2821,  0.1475,  0.1473,  0.1673,  0.1538,  0.1334],
        [ 0.3949,  0.2065,

In [7]:
tol = config.tol_factor


(
    linear_intercept_hat,
    linear_coef_hat,
    lr,
) = training.fit_linear_regression(y, X)

X_for_lr = gs.array(X.reshape(len(X), 1))
y_pred_for_lr = lr.predict(X_for_lr)
y_pred_for_lr = y_pred_for_lr.reshape(y.shape)

(
    geodesic_intercept_hat,
    geodesic_coef_hat,
    gr,
) = training.fit_geodesic_regression(
    y,
    space,
    X,
    tol=tol,
    intercept_hat_guess=linear_intercept_hat,
    coef_hat_guess=linear_coef_hat,
    initialization=config.geodesic_initialization,
    linear_residuals=config.linear_residuals,
)

y.shape:  torch.Size([20, 6])
X.shape:  torch.Size([20])
y.shape:  torch.Size([20, 6])
in loss function
tangent_vec tensor([0.5321, 0.3688, 0.4065, 0.4001, 0.3989, 0.3103])
base_point tensor([1.3170, 0.3761, 0.2441, 0.4892, 0.3427, 0.4203])
intercept belongs to space: tensor(True)
tensor([1.3170, 0.3761, 0.2441, 0.4892, 0.3427, 0.4203])
torch.float64
coef belongs to tangent space: tensor(True)
tensor([0.5321, 0.3688, 0.4065, 0.4001, 0.3989, 0.3103])
torch.float64
X tensor([-0.5000, -0.4474, -0.3947, -0.3421, -0.2895, -0.2368, -0.1842, -0.1316,
        -0.0789, -0.0263,  0.0263,  0.0789,  0.1316,  0.1842,  0.2368,  0.2895,
         0.3421,  0.3947,  0.4474,  0.5000])
X.dtype torch.float64
--> In hyperboloid.py, exp
tangent_vec tensor([[-0.2661, -0.1844, -0.2032, -0.2001, -0.1995, -0.1552],
        [-0.2381, -0.1650, -0.1818, -0.1790, -0.1785, -0.1388],
        [-0.2101, -0.1456, -0.1604, -0.1580, -0.1575, -0.1225],
        [-0.1820, -0.1262, -0.1391, -0.1369, -0.1365, -0.1062],
        

ValueError: Cannot project a vector of norm 0. in the Minkowski space to the hyperboloid

In [ ]:
intercept = gs.array([56.9922, 24.8500, 27.0954, 27.0974, 26.7694, 21.0845])
#coef belongs to tangent space: tensor(True)
coef =  gs.array([2560.6206, 1116.8364, 1217.7700, 1217.8336, 1203.1051,  947.5913])

X =  gs.array([-0.5000, -0.4474, -0.3947, -0.3421, -0.2895, -0.2368, -0.1842, -0.1316,
        -0.0789, -0.0263,  0.0263,  0.0789,  0.1316,  0.1842,  0.2368,  0.2895,
         0.3421,  0.3947,  0.4474,  0.5000])
intercept.dtype

In [ ]:
exp = space.metric.exp(gs.einsum("n,...->n...", X, coef), intercept)
# space.regularize(exp)

In [ ]:
2.8793e+09

In [ ]:
import math
import geomstats.algebra_utils as utils
import geomstats.backend as gs
from geomstats.geometry._hyperbolic import HyperbolicMetric, _Hyperbolic
from geomstats.geometry.base import LevelSet
from geomstats.geometry.minkowski import Minkowski

intercept = gs.array([56.9922, 24.8500, 27.0954, 27.0974, 26.7694, 21.0845])
#coef belongs to tangent space: tensor(True)
coef =  gs.array([2560.6206, 1116.8364, 1217.7700, 1217.8336, 1203.1051,  947.5913])

X =  gs.array([-0.5000, -0.4474, -0.3947, -0.3421, -0.2895, -0.2368, -0.1842, -0.1316,
        -0.0789, -0.0263,  0.0263,  0.0789,  0.1316,  0.1842,  0.2368,  0.2895,
         0.3421,  0.3947,  0.4474,  0.5000])

#exp function in hyperboloid
tangent_vec = gs.einsum("n,...->n...", X, coef)
base_point = intercept

sq_norm_tangent_vec = space.embedding_space.metric.squared_norm(
    tangent_vec
)
sq_norm_tangent_vec = gs.clip(sq_norm_tangent_vec, 0, math.inf)
print("sq_norm_tangent_vec", sq_norm_tangent_vec)
coef_1 = utils.taylor_exp_even_func(
    sq_norm_tangent_vec, utils.cosh_close_0, order=5
)
coef_2 = utils.taylor_exp_even_func(
    sq_norm_tangent_vec, utils.sinch_close_0, order=5
)

exp = gs.einsum("...,...j->...j", coef_1, base_point) + gs.einsum(
    "...,...j->...j", coef_2, tangent_vec
)
print("this exp should be weird", exp)

space.regularize(exp)

In [ ]:
sq_norm = space.embedding_space.metric.squared_norm(coef)

In [ ]:
sq_norm

In [ ]:
exp_norm_0 = gs.array([ 2.8793e+09,  1.2557e+09,  1.3691e+09,  1.3692e+09,  1.3526e+09,
          1.0654e+09])

In [ ]:
space.embedding_space.metric.squared_norm(exp_norm_0)

In [ ]:
gs.all(1.6137e+14)